In [14]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
from poly2trellis import poly2trellis

v=np.array([3,4,5,6,7,8,9,10],dtype=int)
G_D=np.array([[13,17],
     [27,31],
     [53,75],
     [133,171],
     [247,371],
     [561,753],
     [1131,1537],
     [2473,3217]],dtype=int)



(8, 2)


Q2: Minimum Distance & Nearest Neighbors of ZTCC and TBCC (K=64)

In [17]:
%reload_ext autoreload
%autoreload 2
from dmin_nn_calculation import *

K=64

# Create empty lists
ZT_dmin=[]
ZT_NN=[]
TB_dmin=[]
TB_NN=[]


for i in range(len(v)):
    trellis=poly2trellis(1,2,v[i],G_D[i])
    dmin, NN=ZTCC_dmin(K,v[i],trellis)
    ZT_dmin.append(dmin)
    ZT_NN.append(NN)

    dmin, NN=TBCC_dmin(K,v[i],trellis)
    TB_dmin.append(dmin)
    TB_NN.append(NN)


# Display the result
v=[3,4,5,6,7,8,9,10]
data = {
    "v": v,
    "ZT dmin": ZT_dmin,
    "ZT NN": ZT_NN,
    "TB dmin": TB_dmin,
    "TB NN": TB_NN
}


# Create the DataFrame
df = pd.DataFrame(data)
display(HTML(df.to_html(index=False)))



v,ZT dmin,ZT NN,TB dmin,TB NN
3,6,63,6,64
4,7,125,7,128
5,8,62,8,64
6,10,660,10,704
7,10,63,10,64
8,12,667,12,704
9,12,61,12,64
10,14,828,14,896


Q3: Convergence to the RCU (Random-Coding Union) bound 
As compared to zero-terminated convolutional codes (ZTCC), tail-biting (TBCC) increases the rate while degrading performance at low SNR. 
This problem explores the performance of both tail-biting (TB) and zero-terminated (ZT) codes. Continuing with the case of 𝐾 = 64 message bits, for both zero-terminated and tail-biting versions of the rate-1/2 convolutional codes with 𝑣 = 3,4,5, ⋯ ,14 listed in the table of Problem 2, compute the union bound on codeword error rate 𝑃_cw using the “better bound” that requires the 𝑑min calculated in Problem 2, as well as the random coding union (RCU) bound 𝑃_RCU on achievable codeword error rate. 

In [ ]:
import math
import matplotlib.pyplot as plt
from qfunction import Q
from decimal import Decimal, getcontext
# Set precision high enough for your calculations
getcontext().prec = 20

k=1
n=2
K=64
SNR_dB=np.arange(1,8.1,0.1)
SNR=10**(SNR_dB/10)
output_mapping=np.array([0,1,1,2],dtype=int)
ZT_dmin=np.array([6,7,8,10,10,12,12,14],dtype=float)
TB_dmin=np.array([6,7,8,10,10,12,12,14],dtype=float)
Pcw_ZT=np.ones(len(SNR),dtype=float)
Pcw_TB=np.ones(len(SNR),dtype=float)

# Define a small threshold
threshold = 1e-10

plt.figure(figsize=(10, 6))
for i in range(len(v)):
    trellis=poly2trellis(k,n,v[i],G_D[i])

    for j in range(len(SNR)):

        T_W=np.zeros((trellis.numStates,trellis.numStates), dtype=float)
        W=Decimal(math.exp(-SNR[j]/2))

        # fill out T(W): column - current state, row - next state
        for current_state in range(trellis.numStates):
            # input: 0
            next_state=trellis.nextStates[current_state][0]
            T_W[next_state][current_state]=W**output_mapping[trellis.outputs[current_state][0]]

            # input: 1
            next_state=trellis.nextStates[current_state][1]
            T_W[next_state][current_state]=W**output_mapping[trellis.outputs[current_state][1]]
        
        # Convert all elements in T_W to Decimal
        #T_W = np.vectorize(Decimal)(T_W)
        print("T(W) of ", v[i], "and SNR: ", SNR[j], "is ")
        print(T_W)

        # A(W) of ZTCC
        T_W_powered=T_W**(K+v[i])
        A_W_ZTCC=T_W_powered[0][0]-1

        # A(W) of TBCC
        T_W_powered=T_W**K
        A_W_TBCC=np.sum(np.diagonal(T_W_powered))-1

        Pcw_ZT[j]=float(Q(math.sqrt(ZT_dmin[i]) * SNR[j]) * math.exp(ZT_dmin[i] * SNR[j] / 2) * A_W_ZTCC)
        Pcw_ZT[j]=float(Q(math.sqrt(TB_dmin[i]) * SNR[j]) * math.exp(TB_dmin[i] * SNR[j] / 2) * A_W_TBCC)
        # Pcw_TB[j]=Q(math.sqrt(TB_dmin[i])*SNR[j])*math.exp(TB_dmin[i]*SNR[j]/2)*A_W_TBCC

        # Apply threshold to avoid zero results
        Pcw_ZT[j] = max(Pcw_ZT[j], threshold)
        Pcw_TB[j] = max(Pcw_TB[j], threshold)


#print(SNR)
#print(Pcw_ZT)
#print(Pcw_TB)
plt.plot(SNR, Pcw_ZT)
plt.plot(SNR, Pcw_TB)

# Set x and y limits
plt.xlim(1, 8)
plt.ylim(10**-7, 1)

# Set logarithmic scale for y-axis
plt.yscale('log')

# Add labels and title
plt.xlabel('SNR (dB)')
plt.ylabel('Pcw')
plt.title('Pcw vs SNR')
plt.legend()

# Add grid
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Show the plot
plt.show()
        

In [ ]:
from scipy.io import loadmat
SNR_dB=np.arange(1,8.1,0.1)
SNR=10**(SNR_dB/10)
# dmin from v=10 to v=14
ZT_dmin_v10_to_v14=[14,15,16,16,18]
TB_dmin_v10_to_v14=[14,15,16,16,16]

# Load the .mat file
A_W_ZTCC_coeff_v10 = loadmat('/Users/tracychiayu/Documents/231E Project Python ver./AwZTK64v10.mat')
# Extract the array from the dictionary
A_W_ZTCC_coeff_v10 = A_W_ZTCC_coeff_v10['AwZT']
print(A_W_ZTCC_coeff_v10[0].shape)
Pcw_ZT_v10=np.zeros(len(SNR), dtype=float)
ZT_dmin10=14.0
for j in range(len(SNR)):
    W=math.exp(-SNR[j]/2)
    A_W_ZTCC_v10=0.0
    for n in range(1,len(A_W_ZTCC_coeff_v10[0])):
        print("enter the coefficient loop")
        A_W_ZTCC_v10 += float(A_W_ZTCC_coeff_v10[0][n]*(W**n))
    Pcw_ZT_v10[j]=float(Q(math.sqrt(ZT_dmin10 * SNR[j])) * math.exp(ZT_dmin10 * SNR[j] / 2) * A_W_ZTCC_v10)
    print("A(W) ZTCC v=10: ", A_W_ZTCC_v10)
    

plt.semilogy(SNR_dB,Pcw_ZT_v10)
# Add labels and title
plt.xlabel('SNR(dB)')
plt.ylabel('Pcw')
# Show grid
plt.grid(True, which="both", linestyle='--')

# Display the plot
plt.show()

# Explore the data
print(Pcw_ZT_v10)